# Unit Testing Self organising maps using KNN anomaly detection

In [ ]:
import writefile_run as writefile_run

In [ ]:
% matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:

import pandas as pd
import numpy as np
import json
import os
# importing the bayesian changepoint main python file to detect changepoints
from anomaly_detectors.som_knn_detector import som_knn_wrapper as som_wrapper
from anomaly_detectors.utils import reader_helper
from anomaly_detectors.utils import csv_prep_for_reader as csv_helper
from anomaly_detectors.reader_writer import db_properties as db_properties
from anomaly_detectors.reader_writer import writer_configs as writer_configs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# %%writefile_run bayeschangept_sprint1.py -a
'''
Arguments for reader module to get data from opentsdb
This is included for now just for testing, later the main function will take json as direct input
'''

assetno = ['TSFAD_A1']
con = '192.168.2.5:4242'
src_type =  'opentsdb'
param = ['ec2_cpu_utilization_5f5533']
# param = ['ec2_cpu_utilization_ac20cd']

# from_timestamp = 1392388020
# to_timestamp = 1393597320
# param=['ec2_cpu_utilization_5f5533', 'rds_cpu_utilization_cc0c53']
from_timestamp=1392388200
to_timestamp=1393597320
# from_timestamp =1396448940
# to_timestamp = 1397659740

In [ ]:
reader_kwargs= lambda:{
            'assetno':['TSFAD_A1'],
            'from_timestamp':from_timestamp,
            'to_timestamp':to_timestamp,
            'con':con,
            'para_list':param,
            'source_type':src_type,
            'table_name':'',
            'qry_str':'',
            'impute_fill_method':'forward',
            'down_sampling_method':None,
            'down_sampling_window':None,
            'freq':None,
            'resample_fill_method':None,
            'to_resample':None,
            'to_impute':True,
}

model_input_args = lambda :{
    'network_shape':(8,8),
    'input_feature_size':None,
    'time_constant':None,
    'minNumPerBmu':2,
    'no_of_neighbours':3,
    'init_radius':0.4,
    'init_learning_rate':0.01,
    'N':100,    
    'diff_order':1
}

training_args = lambda:{
            'is_train':True,
            'epochs':5,
            'batch_size':4,
            'to_plot':True,
            'test_frac':0.2
        }


        
eval_args = lambda: {
    'model_path':'',
    'to_plot':True,
    'anom_thres':3
}

In [ ]:
keys = list(model_input_args().keys())+list(training_args().keys())+list(eval_args().keys())

In [ ]:
keys

## Test for modes :
#### Testing three different modes of the program
* First training and for testing three different modes are tested and the output is shown below

In [ ]:
def get_kwargs():
    return model_input_args(),training_args(),eval_args()

In [ ]:
reader_kwargs1= reader_kwargs()
json_data = reader_helper.read(reader_kwargs=reader_kwargs1)
model_input_args1,training_args1,eval_args1 = get_kwargs()
training_args1['to_plot'] = True

for i in range(3):
    mode = som_wrapper.mode_options[i]
    print("\nTesting mode option : {}\n".format(mode))
    training_args1['to_plot']=False
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    eval_args1['anom_thres'] = 3
    eval_args1['model_path'] = train_res['models'][0][assetno[0]]
#     json_data = reader_helper.read(reader_kwargs1)
    eval_args1['to_plot']=False
    test_res = som_wrapper.evaluate(**eval_args1,json_data=json_data,mode=som_wrapper.mode_options[i])
    print(test_res)

## Test 1 : 
#### Testing with parameters being empty quotes

In [ ]:
reader_kwargs1= reader_kwargs()
json_data = reader_helper.read(reader_kwargs=reader_kwargs1)
model_input_args1,training_args1,eval_args1 = get_kwargs()

## Testing on model_input_args

In [ ]:
for key in list(model_input_args1.keys()):
    print("\nGiving {} parameter : ''\n".format(key))
    model_input_args1 = model_input_args()
    model_input_args1[key] = ''
    training_args1['to_plot']=False
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    print(train_res)

## Testing on training args

In [ ]:
model_input_args1 = model_input_args()

for key in list(training_args().keys()):
    training_args1 = training_args()
    training_args1[key] = ''
#     eval_args1['to_plot']=False
    training_args1['to_plot']=False
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    print(train_res)

## Testing on evaluation args

In [ ]:
training_args1 = training_args()
model_input_args1 = model_input_args()
training_args1['to_plot']=False
train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
print(train_res)

In [ ]:
for key in list(eval_args().keys()):
    eval_args1 = eval_args()
    eval_args1['model_path'] = train_res['models'][0][assetno[0]]
    eval_args1[key] = ''
#     json_data = reader_helper.read(reader_kwargs1)
    eval_args1['to_plot']=False
    test_res = som_wrapper.evaluate(**eval_args1,json_data=json_data)
    print(test_res)

## Test 2 :
#### Testing missing parameters : 

In [ ]:

   
json_data = reader_helper.read(reader_kwargs=reader_kwargs())
algo_kwargs1 = algo_kwargs()
for key in keys:
    del algo_kwargs1[key]
    res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
    print(res)

## Test 3:
#### Testing parameter type mismatch 

In [ ]:
keys

In [ ]:

json_data = reader_helper.read(reader_kwargs=reader_kwargs())
algo_kwargs1 = algo_kwargs()
val = ['2',4.5,'def']
for i,key in enumerate(keys):
    algo_kwargs1[key]=val[i]
    res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
    print(res)

### User Defined Test Case:
#### Testing the fact that probability threshold must be between 0 and 1

In [ ]:
json_data = reader_helper.read(reader_kwargs=reader_kwargs())
algo_kwargs1 = algo_kwargs()
val = [5,4,100]
for i,key in enumerate(keys):
    algo_kwargs1[key]=val[i]
    res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
    print(res)

## Test 4:
#### Testing Algorithm Logic :
* Running the algorithm for different values of probability thresholds
* so expected o/p behaviour expected is as we increase the probability threshold the no of anomaly detected reduces and reaches zero when set to $1.0$
* Default threshold is $0.5$

In [ ]:


reader_kwargs1 = csv_helper.get_csv_kwargs()
json_data = reader_helper.read(reader_kwargs=reader_kwargs1)
algo_kwargs1 = algo_kwargs()

pthreses = [0.5,0.01,0.99]

for i,pthres in enumerate(pthreses):
    algo_kwargs1['thres_prob']=pthres
    res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
    print(res)

#### Observe the effect of expected_run_length on the algorithm

In [ ]:
reader_kwargs1 = csv_helper.get_csv_kwargs()
json_data = reader_helper.read(reader_kwargs=reader_kwargs1)
algo_kwargs1 = algo_kwargs()
algo_kwargs1['thres_prob'] = 0.5
mean_run_lens = [0,100,10000]
for i,mean_run_len in enumerate(mean_run_lens):
    print('\n Anomaly detection for expected run length  = {}\n'.format(mean_run_len))
    algo_kwargs1['expected_run_length']=mean_run_len
    res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
    print(res)

##### Results:
* As we observed that when expected run length given zero. the algo stops and throws an zero division error as expected/
* Then as we increase it from zero, we observed that interval between changepoints increased

## Test 5:
#### Testing Algorithm tuning :
* Trying to change the parameters of algorithm and observe the results obtained
* Here we try to tune the algo to run faster and as we observe that on increasing expected run length, the sensitivity of detecting changepoints decreases which we saw above case too.

In [ ]:
%%timeit -n 1 -r 1

algo_kwargs1 = algo_kwargs()

    
reader_kwargs1 = csv_helper.get_csv_kwargs(infile='../../dataset/bearings_1.csv',
                                           filename='bearings_1.csv',n_rows=5000,has_time=False)
json_data = reader_helper.read(reader_kwargs=reader_kwargs1)

res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
print(res)

## Test 6:
#### Testing Asset Timeline Logging :

In [ ]:
reader_kwargs1 = csv_helper.get_csv_kwargs()
json_data = reader_helper.read(reader_kwargs1)
algo_kwargs1 = algo_kwargs()
res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
print(res)

!['Asset timeline logging 1'](./atl_test2_bayes.png)
!['Asset timeline logging 2](./atl_test3_bayes.png)

## Test 7:
#### Testing the response from program to follow agreed upon template :

In [ ]:
reader_kwargs1 = csv_helper.get_csv_kwargs()
json_data = reader_helper.read(reader_kwargs1)
algo_kwargs1 = algo_kwargs()
res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
print(res)

## Test 8:
#### Testing No Data exception :
* To get empty dataframe we set from and to timestamp to be not in range of the timestamps in dataset analysed

In [ ]:
reader_kwargs1 = csv_helper.get_csv_kwargs()
reader_kwargs1['from_timestamp'] = int(2**60)
json_data = reader_helper.read(reader_kwargs1)
algo_kwargs1 = algo_kwargs()
res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
print(res)

## Test 9:
#### Testing Database connectivity  exception :
* To test this we edit the db properties and run the algo

In [ ]:
writer_configs.table_name = 'f'

In [ ]:
db_properties.db_connection

### We change the db name and we expect a database error as shown below

In [ ]:
db_properties.db_connection['dbname'] = 'eg'

In [ ]:
reader_kwargs1 = csv_helper.get_csv_kwargs()
json_data = reader_helper.read(reader_kwargs1)
algo_kwargs1 = algo_kwargs()
algo_kwargs1['to_plot'] = False
res = bayeschangept.main(**algo_kwargs1,json_data=json_data,mode=bayeschangept.mode_options[2])
print(res)

### Now we try to change the password of db properties and observe the exception handled

In [ ]:
db_properties.db_connection['password']='fef'

In [ ]:
reader_kwargs1 = csv_helper.get_csv_kwargs()
json_data = reader_helper.read(reader_kwargs1)
algo_kwargs1 = algo_kwargs()
algo_kwargs1['to_plot'] = False
res = bayeschangept.main(**algo_kwargs1,json_data=json_data,mode=bayeschangept.mode_options[2])
print(res)

In [ ]:
%%writefile_run db_properties.py

db_connection = {'dbname': 'Cerebra',
 'host': '127.0.0.1',
 'password': 'givemeachance',
 'port': '5432',
 'user': 'postgres'}

### Now we change the table name in which we are writing, and we observe that relation doesn't exist as seen below

In [ ]:
writer_configs.table_name = 'ffee'

In [ ]:

reader_kwargs1 = csv_helper.get_csv_kwargs()
# reader_kwargs1['from_timestamp'] = int(2**60)
json_data = reader_helper.read(reader_kwargs1)
algo_kwargs1 = algo_kwargs()
algo_kwargs1['to_plot'] = False
res = bayeschangept.main(**algo_kwargs1,json_data=json_data,mode=bayeschangept.mode_options[2])
print(res)

In [ ]:
writer_configs.table_name = 'public.log_asset_timeline'

## Test 10:
#### Testing random exceptions :

In [ ]:
reader_kwargs1 = csv_helper.get_csv_kwargs()
json_data = reader_helper.read(reader_kwargs1)
algo_kwargs1 = algo_kwargs()
algo_kwargs1['to_plot'] = False
res = bayeschangept.main(**algo_kwargs1,json_data=json_data)
print(res)

## Test 11:
#### Testing High Performance :

In [23]:
reader_kwargs1= csv_helper.get_csv_kwargs(infile='../../dataset/bearings.csv',filename='bearings.csv',has_time=False)
json_data = reader_helper.read(reader_kwargs=reader_kwargs1)
model_input_args1,training_args1,eval_args1 = get_kwargs()

                  timestamp      0      1      2      3      4      5      6  \
0 1970-08-07 08:30:00-04:00  0.032 -0.156  0.027  0.027  0.002 -0.122 -0.034   
1 1970-08-07 08:40:00-04:00 -0.071 -0.107  0.000  0.051 -0.178 -0.132 -0.078   
2 1970-08-07 08:50:00-04:00 -0.132 -0.042 -0.044  0.012 -0.093 -0.120 -0.183   
3 1970-08-07 09:00:00-04:00 -0.129 -0.122  0.002  0.037 -0.093 -0.288 -0.156   
4 1970-08-07 09:10:00-04:00  0.022 -0.129 -0.144 -0.088 -0.024 -0.083 -0.129   

       7   assetno  
0 -0.151  TSFAD_A1  
1 -0.051  TSFAD_A1  
2 -0.100  TSFAD_A1  
3 -0.090  TSFAD_A1  
4 -0.098  TSFAD_A1  


In [32]:
%%timeit -n 1 -r 1

training_args1['to_plot']=False
training_args1['epochs']= 10
training_args1['batch_size'] = 32
train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))

eval_args1['anom_thres'] = 3
eval_args1['model_path'] = train_res['models'][0][assetno[0]]
eval_args1['to_plot']=False
test_res = som_wrapper.evaluate(**eval_args1,json_data=json_data,mode=som_wrapper.mode_options[1])
print(test_res)

Data reader initialised 

Getting the dataset from the reader....

Data of Asset no: TSFAD_A1 
               assetno         0         1         2         3         4  \
timestamp                                                                 
18880200000  TSFAD_A1  1.441895 -0.803357  1.352302  1.589216  1.034719   
18880800000  TSFAD_A1  0.277494 -0.160994  1.054745  1.901840 -0.911613   
18881400000  TSFAD_A1 -0.412103  0.691120  0.569836  1.393826  0.007488   
18882000000  TSFAD_A1 -0.378189 -0.357636  1.076786  1.719476  0.007488   
18882600000  TSFAD_A1  1.328847 -0.449402 -0.532229  0.091225  0.753582   

                    5         6         7  
timestamp                                  
18880200000 -0.289267  0.910978 -0.847651  
18880800000 -0.392270  0.224991  0.600962  
18881400000 -0.268666 -1.412023 -0.108858  
18882000000 -1.999118 -0.991077  0.036003  
18882600000  0.112445 -0.570130 -0.079886  

Shape of the Entire dataset : torch.Size([20480, 8])

torch.float64
S

####  We observe that for running oddly $20000$ datapoints with $8$ metric name it takes around $14.7$ seconds to train and test with detect and log mode option on 20000 points with 8 metrics!.

# Conclusion:
* Hence we observe that **Bayesian Changepoint Detection** works well only on level shifts or variational shift datasets over outlier or surge,sag datasets